# Magyar keresztnevek

Próbáljuk meg LogRegr-rel megjósolni a vhányadik karakter alapján, hogy egy magyar keresztnév ffi vagy női. :)

Sass Bálint (joker@nytud.hu)


In [233]:
import numpy as np

# adat:
# wget http://www.nytud.hu/oszt/nyelvmuvelo/utonevek/osszesffi.txt
# wget http://www.nytud.hu/oszt/nyelvmuvelo/utonevek/osszesnoi.txt
# .. és ráadásul utána kézzel ki kell törölni az első sorunkat, bocs.

ffi = np.loadtxt("osszesffi.txt",dtype=str)
noi = np.loadtxt("osszesnoi.txt",dtype=str)

print(ffi)
lffi=len(ffi)
print(lffi)

print(noi)
lnoi=len(noi)
print(lnoi)

['Aba' 'Ab\xe1d' 'Abb\xe1s' ..., 'Zsongor' 'Zsubor' 'Zs\xfclien']
1639
['Ab\xe9lia' 'Abi\xe1ta' 'Abig\xe9l' ..., 'Zsuzska' 'Zs\xfcliett'
 'Zs\xfcsztin']
2175


In [234]:
# baseline = mind női
baseline = lnoi/(lnoi+lffi+0.0)
print(baseline)

0.570267435763


In [235]:
# (tutira nincs 3 betűnél rövidebb név, de ezt akár ellenőrizni is lehetne...)

# * ebben bízunk, hogy mond vmi értelmeset
feature_ind = np.array( [ 0, 1, 2, -3, -2, -1 ] ) # feature-ök: első 3 + utsó 3 betű :)
# eredmény: 65-70% körül
# és tökre nem tuti, hogy nő a tanítóhalmaz méretével -- miért? overfitting?
# hm.. hát lehet, hogy tényleg csak ennyit tud = 3-3 betű alapján nem nagyon lehet megmondani egy névről a nemét!

# * ebben is bízunk
#feature_ind = np.array( [ -2, -1 ] ) # feature-ök: utsó 2 betű :)    
# eredmény: 65-70% körül -- miért ua?
# az eredmény csomószor csontra ua -- a tanítóhalmaz méretétől fgtlenül! (miért?)

# * ez is hátha képes vmire, pl. ha 'a' a vége, akkor női...
#feature_ind = np.array( [ -1 ] ) # 1 db feature: utsó betű :)
# eredmény: 60-65% körül -- romlott, oké.

# * ennek totál homálynak kéne lennie
#feature_ind = np.array( [ 0 ] ) # 1 db feature: első betű :)
# eredmény: 50-60% körül -- oké, rosszabb, kb. a baseline = "mindegyik lány" = 57%

# * ennek is totál homálynak kéne lennie
#feature_ind = np.array( [ 1 ] ) # 1 db feature: 2. betű :)
# eredmény: 50-60% körül -- kb. csak annyira rossz, mint az előző

# az baj, hogy ugyanaz van sokszor? (pl.: 'ia', ka', 'sz', rd'... mint utsó 2 betű)

# -----

feature_cnt = len(feature_ind)

data = np.zeros((lffi+lnoi,feature_cnt+1), dtype=np.uint8) # +1 az osztálynak: ffi=1, noi=2

def str2features(s):
    b = np.fromstring(s, dtype=np.uint8)
    return np.array( [ b[feature_ind] ] )

# hogy lehetne vektorizálni... meg szebben összefűzni a kettőt...
for i, n in enumerate(ffi):
    data[i,0:feature_cnt] = str2features(n)
    data[i,feature_cnt:feature_cnt+1] = 1
    
for i, n in enumerate(noi):
    data[lffi+i,0:feature_cnt] = str2features(n)
    data[lffi+i,feature_cnt:feature_cnt+1] = 2

print(data)

def bytes2str(b):
    return "".join(map(chr, b))

#counts = dict()
#for i in data:
#    j = bytes2str( i[0:-1] )
#    counts[j] = counts.get(j, 0) + 1
#print(counts)

[[ 65  98  97 ...,  98  97   1]
 [ 65  98 225 ..., 225 100   1]
 [ 65  98  98 ..., 225 115   1]
 ..., 
 [ 90 115 117 ..., 107  97   2]
 [ 90 115 252 ..., 116 116   2]
 [ 90 115 252 ..., 105 110   2]]


In [236]:
# adatrandomizálás
np.random.shuffle(data)

print(data)

for i in range(0,9):
    print(data[i],bytes2str(data[i]))

# training és test vágás
train_data = data[0:3000,0:feature_cnt]
train_labels = data[0:3000,feature_cnt:feature_cnt+1]
test_data = data[3000:,0:feature_cnt]
test_labels = data[3000:,feature_cnt:feature_cnt+1]

[[ 75 111 114 ..., 105  97   2]
 [ 67 105 112 ..., 110  97   2]
 [ 90 115 117 ..., 115  97   2]
 ..., 
 [ 77 111 110 ..., 116 105   1]
 [ 83 122 111 ..., 111 110   1]
 [ 70 245  98 ...,  98 101   2]]
(array([ 75, 111, 114, 108, 105,  97,   2], dtype=uint8), 'Korlia\x02')
(array([ 67, 105, 112, 225, 110,  97,   2], dtype=uint8), 'Cip\xe1na\x02')
(array([ 90, 115, 117, 122, 115,  97,   2], dtype=uint8), 'Zsuzsa\x02')
(array([ 76, 105, 100, 105, 100,  97,   2], dtype=uint8), 'Lidida\x02')
(array([ 73, 108, 117, 108, 117, 115,   2], dtype=uint8), 'Ilulus\x02')
(array([ 75, 111, 114,  97, 108, 108,   2], dtype=uint8), 'Korall\x02')
(array([ 82, 111, 100, 105, 111, 110,   1], dtype=uint8), 'Rodion\x01')
(array([ 80, 105, 108, 233, 115, 122,   1], dtype=uint8), 'Pil\xe9sz\x01')
(array([ 65, 122, 105, 122, 105, 122,   1], dtype=uint8), 'Aziziz\x01')


In [237]:
# logistic regression

from sklearn.linear_model import LogisticRegression

for train_size in (50, 500, 1000, 3000):

    regr = LogisticRegression()

    regr.fit(train_data[0:train_size], train_labels[0:train_size].ravel())
    print(regr.score(test_data, test_labels.ravel()))


0.644963144963
0.673218673219
0.674447174447
0.68058968059


In [238]:
nemek = { 1 : 'ffi', 2 : 'noi' }
for i in ( 'Baratán', 'Baldia', 'Finód', 'Finóda', 'Finódia', 'Finódió' ):
    res = regr.predict(str2features(i).reshape(1, -1)) # khm.. warning-ba kiírta, hogy kell ez a reshape...
    nem = nemek[res[0]]
    print(i, nem)

('Barat\xc3\xa1n', 'ffi')
('Baldia', 'noi')
('Fin\xc3\xb3d', 'ffi')
('Fin\xc3\xb3da', 'noi')
('Fin\xc3\xb3dia', 'noi')
('Fin\xc3\xb3di\xc3\xb3', 'ffi')
